In [1]:
from functions import *
import config
import mysql.connector
from mysql.connector import errorcode
import math
import pandas as pd
import numpy as np
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

In [ ]:
cur, cnx = connect_to_db('gender')

In [ ]:
# Creating table for child birth rate
TABLES = {}
TABLES['child_birth_rate'] = ("""
    CREATE TABLE child_birth_rate (
    country varchar(50),
    subgroup varchar(200),
    year int(50),
    unit varchar(50),
    value int(50),
    PRIMARY KEY (country, subgroup, year)
    ) 
    """)

In [ ]:
# Creating table for contraceptive prevalence
TABLES['contraceptive_prevalence'] = ("""
    CREATE TABLE contraceptive_prevalence (
    country varchar(50),
    subgroup varchar(200),
    year int(50),
    unit varchar(50),
    value int(50),
    PRIMARY KEY (country, subgroup, year)
    ) 
    """)

In [ ]:
# Creating table for contraceptive prevalence
TABLES['gender_rural_ratio'] = ("""
    CREATE TABLE gender_rural_ratio (
    country varchar(50),
    subgroup varchar(200),
    year int(50),
    unit varchar(50),
    value int(50),
    PRIMARY KEY (country, subgroup, year)
    ) 
    """)

In [ ]:
# Creating table for gender rural ratio
TABLES['gender_urban_ratio'] = ("""
    CREATE TABLE gender_urban_ratio (
    country varchar(50),
    subgroup varchar(200),
    year int(50),
    unit varchar(50),
    value int(50),
    PRIMARY KEY (country, subgroup, year)
    ) 
    """)

In [ ]:
# Creating table for tertiary education enrollment
TABLES['ter_ed_enrollment'] = ("""
    CREATE TABLE ter_ed_enrollment (
    country varchar(50),
    subgroup varchar(200),
    year int(50),
    unit varchar(50),
    value int(50),
    PRIMARY KEY (country, subgroup, year)
    ) 
    """)

In [ ]:
# Creating table for percentage of women teaching secondary education
TABLES['women_teach_sec'] = ("""
    CREATE TABLE women_teach_sec (
    country varchar(50),
    subgroup varchar(200),
    year int(50),
    unit varchar(50),
    value int(50),
    PRIMARY KEY (country, subgroup, year)
    ) 
    """)

In [ ]:
# Creating table for  percentage of women teaching tertiary education
TABLES['women_teach_ter'] = ("""
    CREATE TABLE women_teach_ter (
    country varchar(50),
    subgroup varchar(200),
    year int(50),
    unit varchar(50),
    value int(50),
    PRIMARY KEY (country, subgroup, year)
    ) 
    """)

In [ ]:
# added to DB
cur.execute(TABLES['child_birth_rate']) 
cur.execute(TABLES['contraceptive_prevalence'])
cur.execute(TABLES['gender_urban_ratio'])
cur.execute(TABLES['gender_rural_ratio'])
cur.execute(TABLES['ter_ed_enrollment'])
cur.execute(TABLES['women_teach_sec'])
cur.execute(TABLES['women_teach_ter'])

In [ ]:
# close cursor
cnx.close()
cur.close() # returned True

Populating our Tables

In [2]:
# child birth rate
def insert_query(table_name):
    """ calls SQL query to insert country, subgroup, year, unit, value into pre-existing table """
    insert_query = f"""
    INSERT IGNORE INTO {table_name} 
    (country, subgroup, year, unit, value)
    VALUES 
    (%s, %s, %s, %s, %s)
    """
    return insert_query


In [3]:
def insert_data(table_name, csv_name):
    cur, cnx = connect_to_db('gender')
    df = pd.read_csv(f'Data To Use/{csv_name}.csv')
    stop = df.loc[df['Country or Area'] == 'fnSeqID'].index[0]
    df2 = df.iloc[0:stop]
    df2['Year'] = pd.to_numeric(df2['Year'], errors = 'coerce')
    df2['Value'] = pd.to_numeric(df2['Value'], errors = 'coerce')
    cur.executemany(insert_query(table_name), get_tuples(df2))
    cnx.commit()
    cnx.close()

In [1]:
def get_tuples(df):
    """takes a dataframe and returns a list of tuples of each row with relevant columns"""
    tuple_list = []
    for index, row in df.iterrows():
        tup = tuple(row[['Country or Area', 'Subgroup', 'Year', 'Unit', 'Value']])
        tuple_list.append((tup[0], 
                           tup[1], 
                           int(tup[2]) if not math.isnan(tup[2]) else None, 
                           tup[3], 
                           int(tup[4]) if not math.isnan(tup[4]) else None
                          ))
    return tuple_list

In [ ]:
get_tuples()

In [7]:
insert_data('child_birth_rate', 'child_birth_rate')
insert_data('contraceptive_prevalence', 'contraceptive_prevalence')
insert_data('gender_urban_ratio', 'gender_urban_ratio')
insert_data('gender_rural_ratio', 'gender_rural_ratio')
insert_data('ter_ed_enrollment', 'ter_ed_enrollment')
insert_data('women_teach_sec', 'women_teach_sec')
insert_data('women_teach_ter', 'women_teach_ter')

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


KeyboardInterrupt: 